# Tech Challenge Fase 3

# Aluno
Klauber Lage - RM358972

Link vídeo no Youtube: https://www.youtube.com/watch?v=sYGzyT-TW40

Link repositório no GitHub: https://github.com/klauberfreitas/pos/tree/main/Fase%203/Tech%20Challenge

# O Problema
O desafio consiste em projetar, testar e implementar um modelo fine tuned usando foudation model. O modelo deve ser treinado usando o dataset da Amazon, AmazonTitles-1.3MM. Este modelo deverá também ser capaz de responder perguntas por meio de integração RAG (Retrieve-and-Generate), usando este mesmo dataset, citando as fontes.

# Tarefas
REQUISITOS DO PROJETO

- Treinar um modelo através de Fine Tuning usando o dataset da Amazon
- Usar o modelo treinado para a integração via RAG
- Usar o dataset da Amazon para o RAG
- Fazer perguntas e obter respostas, incluíndo as fontes


### Referência:
[Documento PDF do Desafio](3IADT - Fase 3 - Tech Challenge.pdf)

---



# Relatório 

# 1. Instalação de Bibliotecas

In [41]:
!pip install pandas numpy scikit-learn transformers sentence-transformers faiss-cpu torch rank-bm25 datasets huggingface_hub

# 2. Carregamento e Limpeza do Dataset

In [42]:
import pandas as pd

# Carregar dados em chunks
def load_data(path, chunksize=10000):
    chunks = pd.read_json(path, lines=True, chunksize=chunksize)
    return pd.concat(chunks)

df_raw = load_data('./data/trn.json')

## Explorando o dataset

In [43]:
df_raw.head(20)

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,0000032034,Adult Ballet Tutu Yellow,,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,0000913154,The Way Things Work: An Illustrated Encycloped...,,"[116, 117, 118, 119, 120, 121, 122]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
4,0001381245,Misty of Chincoteague,,[151],[1.0]
5,0001371045,Hilda Boswell's treasury of children's stories...,,[150],[1.0]
6,0000230022,The Simple Truths of Service: Inspired by John...,,"[184, 185, 186, 187, 188, 189, 190, 191, 192, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
8,0000174076,Evaluating Research in Academic Journals - A P...,,"[106, 208, 209, 210, 211, 212, 213, 214, 215, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
9,0001713086,Dr. Seuss ABC (Dr.Seuss Classic Collection) (S...,,"[260, 261, 262, 263, 264, 265, 266, 267]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"


Podemos perceber a quantidade de nulos. Portanto vamos tratar o dataset, limpando estes valores

## Limpeza do dataset

In [44]:
# Função para limpar o dataset
def clean_dataset(df):
    # Remove registros com titles nulos ou vazios
    df = df[df['title'].notna() & (df['title'].str.strip() != '')]
    
    # Remove registros com contents nulos ou vazios
    df = df[df['content'].notna() & (df['content'].str.strip() != '')]
    
    # Remove duplicados, conforme podemos ver acima
    cols_to_check = ['title']

    cols_to_check.append('content')
    
    df = df.drop_duplicates(subset=cols_to_check)
    
    return df

# Chama a função que limpa o dataset
df = clean_dataset(df_raw)

print(f"Dados após limpeza: restaram {len(df)} de {len(df_raw)} registros")

Dados após limpeza: restaram 1367131 de 2248619 registros


# 3. Preparação dos Dados para Fine Tuning

In [45]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Divisão em treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Conversão para o formato do Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Função para tokenizar e formatar os dados de entrada
def preprocess(examples):
    questions = examples['content']
    contexts = examples['title']
    targets = examples['title']
    
    # Cria as entradas para o tokenizer combinando o título com o content
    inputs = [f"question: {question} context: {context}" for question, context in zip(questions, contexts)]
    
    # Tokenizar e formatar as entradas do modelo
    model_inputs = tokenizer(
        inputs,  # Perguntas e Contextos
        max_length=512,  # Define o comprimento máximo das sequências de tokens
        truncation=True,  # Trunca as sequências que excedem o comprimento máximo
        padding="max_length"  # Adiciona padding para que todas as sequências tenham o mesmo comprimento
    )
    
    # Tokenizar targets
    labels = tokenizer(
        targets,  # Títulos
        max_length=128,  # Define o comprimento máximo das sequências de tokens
        truncation=True,  # Trunca as sequências de target que excedem o comprimento máximo
        padding="max_length"  # Adiciona padding para que todas as sequências de target tenham o mesmo comprimento
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Aplicar pré-processamento
tokenized_train = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_test = test_dataset.map(
    preprocess,
    batched=True,
    remove_columns=test_dataset.column_names
)

Map: 100%|██████████| 273427/273427 [00:38<00:00, 7126.69 examples/s]


# 4. Configurações do treinamento

In [46]:
# Variáveis que controlam o comportamento do Fine Tuning e do RAG

# Número de épocas do treinamento
EPOCHS=5

# Definido para o que a minha VRAM aguenta
TRAIN_BATCH_SIZE=8

# Batch efetivo
ACCUMULATION_STEPS=2

# Número máximo de steps do fine tuning
# MAX_STEPS=(len(train_dataset) // TRAIN_BATCH_SIZE) * EPOCHS # Este seria o valor ideal, porém levará muito para terminar durante o exemplo
MAX_STEPS=20

# Número de documentos do dataset a serem usados. Esou usando apenas 500 amostras devido ao alto consumo de memória e CPU, o que estava levando o PC a travar
DOCUMENTS=500

# Top K de resultados para o retriever
K=5

In [47]:
import shutil

# Remove o conteúdo da pasta results para evitar erros de pasta já existente
try:
    shutil.rmtree(f"./results/checkpoint-{MAX_STEPS}")
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

In [48]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Carrega o modelo T5 pré-treinado da Google
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Configuração de treino
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # Pasta para salvar os resultados
    evaluation_strategy="no",  # Desliga a avaliação durante o treino já que estou limitando a 500 documentos e para economizar recursos
    learning_rate=2e-4,  # Aumenta taxa de aprendizado, o ideal é entre 1e-4 a 3e-4
    per_device_train_batch_size=TRAIN_BATCH_SIZE,  # Definido para o que a minha VRAM aguenta
    gradient_accumulation_steps=ACCUMULATION_STEPS,  # Batch efetivo de TRAIN_BATCH_SIZE*ACCUMULATION_STEPS=16
    weight_decay=0.01,  # Decaimento de peso para regularização
    save_total_limit=1,  # Limita o número de checkpoints salvos
    num_train_epochs=EPOCHS,  # Número de épocas de treinamento
    fp16=True,  # Usa precisão mista para reduzir uso de memória (meu pc estava travando)
    logging_steps=100,  # Frequência de logging
    max_steps=MAX_STEPS,  # Limita o número total de passos
    optim="adafactor",  # Optimizer mais leve
    gradient_checkpointing=True,  # Reduz uso de VRAM
)

# Dataset de treinamento e de avaliação
train_dataset = tokenized_train.select(range(DOCUMENTS))  
eval_dataset = tokenized_test.select(range(DOCUMENTS)) 

# Data Collator para preparação dos dados
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Tokenizador a ser usado
    model=model,  # Modelo a ser usado (que é o flan-t5-base)
    padding=True  # Adiciona padding para uniformizar o tamanho das sequências (garantindo que todas as entradas tenham o mesmo comprimento) além de acelerar o treinamento
)

# Cria o Trainer
trainer = Seq2SeqTrainer(
    model=model,  # Modelo a ser treinado (que é o flan-t5-base)
    args=training_args,  # Argumentos de treinamento
    train_dataset=train_dataset,  # Conjunto de dados de treinamento
    eval_dataset=eval_dataset,  # Conjunto de dados de avaliação
    data_collator=data_collator,  # Data collator para preparação dos dados
    tokenizer=tokenizer,  # Tokenizador a ser usado
)

# Inicia o treinamento
trainer.train()

c:\Users\kake\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\kake\AppData\Local\Temp\ipykernel_16072\616683413.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss


TrainOutput(global_step=20, training_loss=13.435350036621093, metrics={'train_runtime': 364.9294, 'train_samples_per_second': 0.877, 'train_steps_per_second': 0.055, 'total_flos': 219122352783360.0, 'train_loss': 13.435350036621093, 'epoch': 0.6349206349206349})

# 5. Carregamento do modelo treinado e Estruturas de Geração de Respostas

In [49]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration

# Configura o caminho para o modelo fine tuned
model_path = f"./results/checkpoint-{MAX_STEPS}"

# Carrega o modelo e o tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path).to("cpu")

documents = (df['title'] + ": " + df['content']).tolist()

# Classe para criar a base do retriever
class SemanticRetriever:
    def __init__(self, documents, batch_size=256):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2').to(self.device)
        self.documents = documents
        self.batch_size = batch_size
        self.embeddings = self.encode_documents(documents)

    #  Calcula e armazena os embeddings dos documentos retornando em um único tensor
    def encode_documents(self, documents):
        embeddings = []
        for i in range(0, len(documents), self.batch_size):
            batch = documents[i:i + self.batch_size]
            batch_embeddings = self.encoder.encode(
                batch,
                convert_to_tensor=True,
                normalize_embeddings=True
            )
            embeddings.append(batch_embeddings)
        return torch.cat(embeddings)

    # Recupera os documentos mais relevantes para as perguntas que forem feitas
    def retrieve(self, query, k=5):
        query_emb = self.encoder.encode(
            query,
            convert_to_tensor=True,
            normalize_embeddings=True
        )
        scores = torch.mm(self.embeddings, query_emb.unsqueeze(0).T)
        top_indices = torch.topk(scores.flatten(), k).indices
        return [self.documents[i] for i in top_indices]

# Função para gerar as respostas de embeddings
def generate_answer(question):
    contexts = retriever.retrieve(question, k=K)

    # Template de prompt otimizado
    context_str = "\n".join([f"- {context}" for context in contexts])
    instruction = f"Generate a comprehensive answer using only the following context: Context: {context_str} Question: {question} Answer:"

    # Tokeniza o input, convertendo em tensores
    inputs = tokenizer(
        instruction,
        return_tensors="pt",  # Retorna tensores do PyTorch
        max_length=1024,      # Define a sequência máxima de tokens
        truncation=True       # Trunca sequências de tokens que excedem o comprimento máximo
    ).to(model.device)     

    # Gera a resposta com base nos inputs já tokenizados
    outputs = model.generate(
        **inputs,                # Passa os inputs tokenizados para o modelo
        max_new_tokens=256,      # Define o número máximo de novos tokens a serem gerados
        num_beams=7,             # Utiliza beam search para melhorar qualidade da geração
        no_repeat_ngram_size=3,  # Evita a repetição de n-gramas
        early_stopping=True,     # Interrompe a geração assim que a resposta estiver completa
        temperature=0.7          # Controla a aleatoriedade da geração (valores mais baixos tornam a saída mais determinística)
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Configura o retriever
retriever = SemanticRetriever(documents[:DOCUMENTS])

# 6. Processamento de Perguntas e Respostas

In [50]:
import json

# Função para carregar as perguntas do arquivo perguntas.json
def load_questions_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Função para processar as perguntas e respostas
def process_questions(file_path):
    questions_data = load_questions_from_json(file_path)
    
    for item in questions_data:
        question = item['question']
        expected_answer = item['expected']
        
        resposta = generate_answer(question)
        context = retriever.retrieve(question, k=3)
        accurate = resposta == expected_answer
        
        print(f"Pergunta: {question}")
        print(f"Resposta gerada: {resposta}")
        print(f"Resposta esperada: {expected_answer}")
        print(f"Contexto: {context}")
        print(f"Resposta correta: {accurate}")
        print("-" * 50)
      


In [51]:
file_path = './perguntas.json'
process_questions(file_path)

c:\Users\kake\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Pergunta: Where Jill Barklem was born?
Resposta gerada: Epping
Resposta esperada: Epping
Contexto: ["Nice for Mice: Jill Barklem was born in Epping in 1951. After an accident when she was thirteen, Jill was unable to take part in PE or games at school and instead developed her talent for drawing and art. On leaving school, she studied illustration at St Martin's in London. Jill is now a full-time illustrator, working on the series of Brambly Hedge books.", 'Dorothy Rowe\'s Guide to Life: Dorothy Rowe was born in Australia in 1930, and worked as a teacher and child psychologist before coming to England, where she obtained her PhD at Sheffield University. From 1972 until 1986 she was head of Clinical Psychology. She is now engaged in writing, lecturing and research, and is world-renowned for her work on how we communicate and why we suffer. Her books include "Wanting Everything\', "Beyond Fear\' and "Time On Our Side\'.', 'Animal Wisdom: Jessica Palmer is descended from Dakota woodland S

In [52]:
file_path = './perguntas2.json'
process_questions(file_path)

Pergunta: Who wrote The Very Bad Bunny?
Resposta gerada: Marilyn Sadler
Resposta esperada: Marilyn Sadler
Contexto: ['The Very Bad Bunny (Beginner Series): By Marilyn Sadler, Illustrated by Roger Bollen', 'Evil Under the Sun: Complete &amp; Unabridged: "You can\'t go wrong with this one" Books "She springs her secret like a land mine" Times Literary Supplement', 'Fix-it Duck (Duck in the Truck): The trouble begins with a leaky roof in the sequel to Duck in a Truck, Fix-It Duck by Jez Alborough, and follows the, er, handy quacker on a series of missteps.Copyright 2002 Cahners Business Information, Inc.--This text refers to theHardcoveredition.']
Resposta correta: True
--------------------------------------------------
Pergunta: Who is in a distant, timeless place?
Resposta gerada: J.R. Tolkien
Resposta esperada: The Prophet
Contexto: ["Ten Thousand Miles Without a Cloud: 'Packed with erudition and perception...it is also honest, sensitive, entirely without ego...ultimately, a meditation

# Debug Rápido

In [54]:
# Debug
question = "What is the best encyclopedia of techonology?" 

context = retriever.retrieve(question, k=3)
print("Contextos relevantes:", context)

answer = generate_answer(question)
print(f"Resposta: {answer}")

Contextos relevantes: ['The Tao of Physics (Flamingo): "A brilliant best seller. . . . Lucidly analyzes the tenets of Hinduism, Buddhism, and Taoism to show their striking parallels with the latest discovery in cyclotrons."--"New York" magazine "A pioneering book of real value and wide appeal."--"Washington Post""I have been reading the book with amazement and the greatest interest, recommending it to everyone I meet and, as often as possible, in my lectures. I think you have done a magnificent and extremely important job."--"Joseph Campbell"', 'Encyclopedia of Essential Oils: The complete guide to the use of aromatic oils in aromatherapy, herbalism, health and well-being.: &#x201C;at last a clear and systematic distillation of useful information about a truly comprehensive spectrum of essential oils and absolutes.&#x201D; John Steele, American Aromatherapy Association.&#x201C;A comprehensive and timely contribution to aromatherapy, herbalism and the whole field of holistic health care